# Выполнил Лялин Илья Евгеньевич ББМО-02-24

**1)Копируем проект**

In [1]:
!git clone https://github.com/ewatson2/EEL6812_DeepFool_Project

Cloning into 'EEL6812_DeepFool_Project'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 96 (delta 2), reused 1 (delta 1), pack-reused 93 (from 1)
Receiving objects: 100% (96/96), 33.99 MiB | 14.70 MiB/s, done.
Resolving deltas: 100% (27/27), done.


**2) Меням директорию на папку с проектом**

In [2]:
%cd EEL6812_DeepFool_Project

/content/EEL6812_DeepFool_Project


**3) Импортируем библиотеки**

In [3]:
import numpy as np # предоставляет поддержку многомерных массивов и математических функций
import json # текстовый формат для хранения и передачи структурированных данных
import torch # PyTorch - фреймворк для машинного обучения с автоматическим дифференцированием

# - DataLoader: класс для загрузки данных батчами (пакетами)
# - random_split: функция для случайного разделения набора данных на части
from torch.utils.data import DataLoader, random_split

# - datasets: содержит популярные наборы данных (MNIST, CIFAR, ImageNet и др.)
# - models: предоставляет предобученные архитектуры нейронных сетей (ResNet, VGG, etc.)
from torchvision import datasets, models

# Импорт модуля transforms из torchvision для преобразований изображений:
# содержит функции для аугментации и предобработки данных (изменение размера, нормализация и т.д.)
from torchvision.transforms import transforms

In [4]:
# - FC_500_150: полносвязная сеть со слоями 500 и 150 нейронов
# - LeNet_CIFAR: архитектура LeNet, адаптированная для датасета CIFAR
# - LeNet_MNIST: архитектура LeNet, адаптированная для датасета MNIST
# - Net: базовая или кастомная архитектура нейронной сети
from models.project_models import FC_500_150, LeNet_CIFAR, LeNet_MNIST, Net

# - get_clip_bounds: функция для получения границ обрезки значений (например, для ограничения пикселей)
# - evaluate_attack: функция для оценки эффективности атаки на модель
# - display_attack: функция для визуализации результатов атаки
from utils.project_utils import get_clip_bounds, evaluate_attack, display_attack

**4) Устанавливаем случайное рандомное значение**
Для rand_seed = порядковый номер студента в таблице

In [5]:
rand_seed = 6

In [6]:
# для numpy
np.random.seed(rand_seed)
# для torch
torch.manual_seed(rand_seed)

Среда выполнения сменяна на T4 GPU.

**5) Загружаем датасет MNIST с параметрами**

In [7]:
# Определение параметров нормализации для MNIST
mnist_mean = 0.5  # среднее значение для нормализации
mnist_std = 0.5   # стандартное отклонение для нормализации
mnist_dim = 28    # размер изображения MNIST (28x28 пикселей)

# Вычисление минимальных и максимальных допустимых значений после нормализации
# (важно для adversarial attacks - чтобы не выходить за допустимые границы пикселей)
mnist_min, mnist_max = get_clip_bounds(mnist_mean, mnist_std, mnist_dim)

# Определение устройства для вычислений: GPU если доступен, иначе CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Перенос граничных значений на выбранное устройство (для совместимости с тензорами)
mnist_min = mnist_min.to(device)
mnist_max = mnist_max.to(device)

# Базовые преобразования для тестовых данных:
# - ToTensor(): конвертация в тензор и нормализация в [0,1]
# - Normalize(): стандартизация с заданными mean и std
mnist_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mnist_mean, std=mnist_std)
])

# Расширенные преобразования для тренировочных данных (с аугментацией):
# - RandomHorizontalFlip(): случайное горизонтальное отражение (для MNIST не всегда уместно)
# - Остальные преобразования аналогичны базовым
mnist_tf_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=mnist_mean, std=mnist_std)
])

# Обратные преобразования для денормализации изображений:
# Восстанавливает оригинальные значения пикселей после нормализации
mnist_tf_inv = transforms.Compose([
    transforms.Normalize(mean=0.0, std=1.0 / mnist_std),  # отмена std
    transforms.Normalize(mean=-mnist_mean, std=1.0)       # отмена mean
])

# Загрузка тренировочного набора MNIST с аугментацией:
# - root: путь к папке с данными
# - train=True: загрузка тренировочной части
# - download=True: скачивание если данных нет локально
# - transform=mnist_tf_train: применение аугментированных преобразований
mnist_temp = datasets.MNIST(root='datasets/mnist', train=True, download=True, transform=mnist_tf_train)

# Разделение на тренировочную и валидационную выборки:
# 50000 образцов для тренировки, 10000 для валидации
mnist_train, mnist_val = random_split(mnist_temp, [50000, 10000])

# Загрузка тестового набора MNIST с базовыми преобразованиями
mnist_test = datasets.MNIST(root='datasets/mnist', train=False, download=True, transform=mnist_tf)

100%|██████████| 9.91M/9.91M [00:01<00:00, 5.10MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 132kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.26MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 13.0MB/s]


**6) Загружаем датасет CIFAR-10**

In [8]:
# Параметры нормализации для CIFAR-10 (указаны для каждого из 3 каналов RGB)
cifar_mean = [0.491, 0.482, 0.447]  # средние значения по каналам R, G, B
cifar_std = [0.202, 0.199, 0.201]   # стандартные отклонения по каналам
cifar_dim = 32  # размер изображений CIFAR-10 (32x32 пикселя)

# Вычисление граничных значений после нормализации (для adversarial attacks)
cifar_min, cifar_max = get_clip_bounds(cifar_mean, cifar_std, cifar_dim)

# Определение вычислительного устройства (GPU/CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Перенос граничных значений на выбранное устройство
cifar_min = cifar_min.to(device)
cifar_max = cifar_max.to(device)

# Базовые преобразования для тестовых данных:
# - ToTensor(): конвертация в тензор [0,1]
# - Normalize(): стандартизация с заданными параметрами
cifar_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=cifar_mean, std=cifar_std)
])

# Расширенные преобразования для тренировочных данных с аугментацией:
# - RandomCrop(): случайное обрезание с padding=4 (увеличивает разнообразие)
# - RandomHorizontalFlip(): случайное горизонтальное отражение
# - Остальные преобразования аналогичны базовым
cifar_tf_train = transforms.Compose([
    transforms.RandomCrop(size=cifar_dim, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=cifar_mean, std=cifar_std)
])

# Обратные преобразования для денормализации изображений:
# Первый шаг: отмена нормализации по std (деление на std)
# Второй шаг: отмена нормализации по mean (вычитание mean)
cifar_tf_inv = transforms.Compose([
    transforms.Normalize(mean=[0.0, 0.0, 0.0], std=np.divide(1.0, cifar_std)),
    transforms.Normalize(mean=np.multiply(-1.0, cifar_mean), std=[1.0, 1.0, 1.0])
])

# Загрузка тренировочного набора CIFAR-10 с аугментацией
cifar_temp = datasets.CIFAR10(root='datasets/cifar-10', train=True, download=True, transform=cifar_tf_train)

# Разделение на тренировочную (40000) и валидационную (10000) выборки
cifar_train, cifar_val = random_split(cifar_temp, [40000, 10000])

# Загрузка тестового набора CIFAR-10 с базовыми преобразованиями
cifar_test = datasets.CIFAR10(root='datasets/cifar-10', train=False, download=True, transform=cifar_tf)

# Список названий классов CIFAR-10 в правильном порядке
cifar_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

100%|██████████| 170M/170M [00:19<00:00, 8.96MB/s]


**7) Настроим и загрузим DataLoader**

In [9]:
# Размер батча - количество образцов, обрабатываемых за одну итерацию
batch_size = 64

# Количество процессов для параллельной загрузки данных
workers = 4

# DataLoader для тренировочных данных MNIST:
# - shuffle=True: случайное перемешивание данных перед каждой эпохой
# - num_workers=4: 4 процесса для параллельной загрузки данных
mnist_loader_train = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=workers)

# DataLoader для валидационных данных MNIST:
# - shuffle=False: данные не перемешиваются (для стабильной оценки)
mnist_loader_val = DataLoader(mnist_val, batch_size=batch_size, shuffle=False, num_workers=workers)

# DataLoader для тестовых данных MNIST:
# - shuffle=False: сохранение порядка для корректной оценки результатов
mnist_loader_test = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=workers)

# DataLoader для тренировочных данных CIFAR-10:
# - shuffle=True: перемешивание для лучшего обучения
cifar_loader_train = DataLoader(cifar_train, batch_size=batch_size, shuffle=True, num_workers=workers)

# DataLoader для валидационных данных CIFAR-10:
# - shuffle=False: стабильная оценка на валидации
cifar_loader_val = DataLoader(cifar_val, batch_size=batch_size, shuffle=False, num_workers=workers)

# DataLoader для тестовых данных CIFAR-10:
# - shuffle=False: сохранение порядка тестовых образцов
cifar_loader_test = DataLoader(cifar_test, batch_size=batch_size, shuffle=False, num_workers=workers)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


**Параметры:**

batch_size=64: Оптимальный баланс между скоростью и стабильностью обучения

shuffle=True для тренировки: Предотвращает запоминание порядка данных моделью

shuffle=False для валидации/теста: Обеспечает воспроизводимость результатов

num_workers=4: Ускоряет загрузку данных за счет параллелизма

**Преимущества:**

Эффективное использование памяти: Данные загружаются батчами, а не все сразу

Ускорение обучения: Parallel data loading пока модель обрабатывает предыдущий батч

Автоматическое перемешивание: Улучшает качество обучения

Удобство итерации: Простой цикл for по батчам

**8) Оценка стойкости модели LeNet к FGSM и DeepFool атакам**

In [10]:
# Установка величины возмущения (epsilon) для FGSM атаки
# eps=0.6 - достаточно большая величина для MNIST (пиксели в диапазоне [0,1])
fgsm_eps = 0.6

# Создание экземпляра модели LeNet для MNIST и перенос на выбранное устройство
model = LeNet_MNIST().to(device)

# Загрузка предобученных весов для модели
# map_location=torch.device('cpu') - гарантирует загрузку на CPU даже если веса были сохранены на GPU
model.load_state_dict(torch.load('weights/clean/mnist_lenet.pth', map_location=torch.device('cpu')))

# Оценка уязвимости модели к FGSM атаке
evaluate_attack(
    'mnist_lenet_fgsm.csv',     # имя файла для сохранения результатов
    'results',                   # папка для сохранения
    device,                      # вычислительное устройство
    model,                       # тестируемая модель
    mnist_loader_test,          # загрузчик тестовых данных
    mnist_min, mnist_max,       # границы значений пикселей
    fgsm_eps,                   # параметр атаки (epsilon)
    is_fgsm=True                # флаг указывающий на тип атаки (FGSM)
)

# Печать пустой строки для разделения выводов
print('')

# Параметры для DeepFool атаки:
deep_args = [64, 10, 0.02, 100]
# Вероятно: [max_iterations, num_classes, overshoot, step_size] или подобные параметры

# Оценка уязвимости модели к DeepFool атаке
evaluate_attack(
    'mnist_lenet_deepfool.csv', # имя файла для сохранения результатов
    'results',                   # папка для сохранения
    device,                      # вычислительное устройство
    model,                       # тестируемая модель
    mnist_loader_test,          # загрузчик тестовых данных
    mnist_min, mnist_max,       # границы значений пикселей
    deep_args,                  # параметры DeepFool атаки
    is_fgsm=False               # флаг указывающий, что это не FGSM атака
)

# Если используется GPU, очищаем кеш памяти для предотвращения утечек памяти
if device.type == 'cuda':
    torch.cuda.empty_cache()

FGSM Test Error : 87.89%
FGSM Robustness : 4.58e-01
FGSM Time (All Images) : 0.29 s
FGSM Time (Per Image) : 28.86 us

DeepFool Test Error : 98.74%
DeepFool Robustness : 9.64e-02
DeepFool Time (All Images) : 193.32 s
DeepFool Time (Per Image) : 19.33 ms


Типы атак и их особенности:
FGSM (Fast Gradient Sign Method):
**Текст, выделенный полужирным шрифтом**
Быстрая однострочная атака

Использует градиент функции потерь

Контролируется параметром epsilon

is_fgsm=True

**DeepFool:**

Итеративная атака, находит минимальное возмущение

Обычно более эффективна чем FGSM

Требует нескольких параметров

is_fgsm=False

**9) Оценка стойкости модели FC к FGSM и DeepFool атакам**

In [11]:
# Установка величины возмущения для FGSM атаки
# eps=0.2 - меньшая величина по сравнению с LeNet (0.6), возможно из-за разной архитектуры
fgsm_eps = 0.2

# Создание экземпляра полносвязной сети FC_500_150 и перенос на устройство
model = FC_500_150().to(device)

# Загрузка предобученных весов для полносвязной модели
# map_location=device - загрузка напрямую на выбранное устройство (CPU/GPU)
model.load_state_dict(torch.load('weights/clean/mnist_fc.pth', map_location=device))

# Оценка уязвимости полносвязной модели к FGSM атаке
evaluate_attack(
    'mnist_fc_fgsm.csv',        # файл для результатов FGSM атаки на FC сеть
    'results',                   # папка для сохранения
    device,                      # вычислительное устройство
    model,                       # полносвязная модель FC_500_150
    mnist_loader_test,          # загрузчик тестовых данных MNIST
    mnist_min, mnist_max,       # границы значений пикселей
    fgsm_eps,                   # параметр атаки (меньший чем для LeNet)
    is_fgsm=True                # флаг FGSM атаки
)

print('')  # Разделитель вывода

# Параметры для DeepFool атаки (аналогичные предыдущему тесту)
deep_args = [64, 10, 0.02, 100]

# Оценка уязвимости полносвязной модели к DeepFool атаке
evaluate_attack(
    'mnist_fc_deepfool.csv',    # файл для результатов DeepFool атаки на FC сеть
    'results',                   # папка для сохранения
    device,                      # вычислительное устройство
    model,                       # полносвязная модель FC_500_150
    mnist_loader_test,          # загрузчик тестовых данных MNIST
    mnist_min, mnist_max,       # границы значений пикселей
    deep_args,                  # параметры DeepFool атаки
    is_fgsm=False               # флаг указывающий на DeepFool атаку
)

# Очистка кеша GPU если используется видеокарта
if device.type == 'cuda':
    torch.cuda.empty_cache()

FGSM Test Error : 87.08%
FGSM Robustness : 1.56e-01
FGSM Time (All Images) : 0.15 s
FGSM Time (Per Image) : 14.99 us

DeepFool Test Error : 97.92%
DeepFool Robustness : 6.78e-02
DeepFool Time (All Images) : 141.81 s
DeepFool Time (Per Image) : 14.18 ms



**Назначение этого теста:**

Сравнение уязвимости: Оценка того, какая архитектура более устойчива к атакам

Анализ влияния архитектуры: Полносвязные vs сверточные сети

Настройка параметров атак: Разные epsilon значения для разных моделей

Сбор сравнительных данных: Для последующего анализа эффективности атак на разные архитектуры